<a href="https://colab.research.google.com/github/s99436q/RNN/blob/master/novel_JS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set Data

###Load data

In [0]:
%tensorflow_version 2.x
%load_ext tensorboard
import tensorflow as tf
print(tf.__version__)
#from google.colab import files
#uploaded = files.upload()
from google.colab import drive
import os
drive.mount('/content/gdrive/')
os.chdir("/content/gdrive/My Drive/Colab Notebooks/Novel/novels")
#os.getcwd()
#!ls
text=open("novel2.txt", encoding="utf-8").read()
print(f"天龍八部小說共有 {len(text)} 中文字,{len(set(text))} 個獨一無二的字")
print(text[100331:100499])

2.2.0
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/
天龍八部小說共有 1266445 中文字,4401 個獨一無二的字
“媽媽，媽媽，我要媽媽！”葉二孃拍著他哄道：“乖孩子，我是你媽媽。”那小兒越哭越響，叫道：“我要媽媽，我要媽媽，你不是我媽媽。”葉二孃輕輕搖幌他身子，雖起兒哥來：“搖搖搖，搖到外婆橋，外婆叫我好寶寶……”那小兒仍是哭叫不休。

    南海鱷神聽得甚是煩躁，喝道：“你哄什麼？要弄死他，乘早弄死了吧。”

    葉二孃臉上笑眯眯地，不


### Prepare data


In [0]:
#intial tokenizor 
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=None,char_level=True,filters='')
#read the text
tokenizer.fit_on_texts(text)
#transfer text to int
text_as_int = tokenizer.texts_to_sequences([text])[0] #return text=[[...]]
#transfer text_as_int to tensor
text = tf.data.Dataset.from_tensor_slices(text_as_int)
#make mappinf between char and index
idx2char=tokenizer.index_word
char2idx=tokenizer.word_index

vocab_size=len(set(text_as_int))

# each seq to input/output pairs
def build_pairs(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

def dataset(batch_size,seq_len,steps_per_epoch):
    #divide the novel text to small pieces of seqs
    seqs = text.batch(seq_len + 1,drop_remainder=True)
    #generate input/output pairs for each seq, shffle them, batch them
    ds=seqs.map(build_pairs).batch(batch_size,drop_remainder=True)
    rds=seqs.map(build_pairs).shuffle(steps_per_epoch).batch(batch_size,drop_remainder=True)
    return ds,rds

# Set Model


### Build model

In [0]:
def build_model(embedding_dim,lstm_units,vocab_size,batch_size):
      #Make a LSTM model
      model = tf.keras.Sequential()
      #Word embedding: map a chinese char to high dimension vectors
      model.add(
        tf.keras.layers.Embedding(
          input_dim=vocab_size, 
          output_dim=embedding_dim,
          batch_input_shape=[batch_size, None]
        )
      )

      #LSTM: process the sequence of data
      model.add(
        tf.keras.layers.LSTM(
          units=lstm_units, 
          return_sequences=True, 
          stateful=True, 
          recurrent_initializer='glorot_uniform'
        )
      )

      #Dense layer: predict the probability of each chinese char
      model.add(
        tf.keras.layers.Dense(vocab_size)
      )

      model.summary()
      return model


### Compile and fit model

In [0]:
#loss function and optimizer
def loss(y_true, y_pred):
    return tf.keras.losses.sparse_categorical_crossentropy(
          y_true, y_pred, from_logits=True
    ) #becase our model return logits, so set from_logits flag

def compile_model(model,learning_rate):
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), 
        loss=loss
    )
def fit_model(model,x,epochs,callbacks): model.fit(x=x,epochs=epochs,callbacks=[callbacks])

###Checkpoints

In [0]:
def get_modelID(lr,seq_len,batch_size,embedding_dim,lstm_units,epochs,data_chuck):  
    return f'{lr}seq{seq_len}batch{batch_size}_embed{embedding_dim}unit{lstm_units}_epoch{epochs}_{data_chuck}shuffle'

def get_checkpoints(lr,seq_len,batch_size,embedding_dim,lstm_units,epochs,data_chuck):
    model_id=get_modelID(lr,seq_len,batch_size,embedding_dim,lstm_units,epochs,data_chuck)
    return "./train/"+model_id+"/checkpoint"

def set_checkpoints(lr,seq_len,batch_size,embedding_dim,lstm_units,epochs,data_chuck):
    model_id=get_modelID(lr,seq_len,batch_size,embedding_dim,lstm_units,epochs,data_chuck)
    log_dir = "./logs/" + model_id
    checkpoint_filepath="./train/"+model_id+"/checkpoint"

    callbacks = [
        tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1),
        tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath,save_weights_only=True
                                           ,mode='min',monitor='loss'
                                           ,save_best_only=True,verbose=1),
      
        tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3),
    ]

    return callbacks



# Training 

In [0]:
def call_model(lr,seq_len,batch_size,embedding_dim,lstm_units,epochs,data_chuck):
    steps_per_epoch=len(text_as_int)//seq_len

    model=build_model(embedding_dim,lstm_units,vocab_size,batch_size)
    compile_model(model,lr)
    data=dataset(batch_size,seq_len,steps_per_epoch)
    callbacks=set_checkpoints(lr, seq_len, batch_size,embedding_dim,lstm_units,epochs,data_chuck)
    fit_model(model,data[data_chuck],epochs,callbacks)


# Predict

In [0]:
def call_predict(embedding_dim,lstm_units,checkpoint_filepath,temperature,num_generate,start_string):
    
    model=build_model(embedding_dim,lstm_units,vocab_size,batch_size=1)
    model.load_weights(checkpoint_filepath)
    model.build(tf.TensorShape([1,None]))
    
    result=generate_text(model,temperature,start_string,num_generate)
    print(result)
    #with open('./result.txt', 'a') as file:
     #     file.write(result)

In [0]:
def generate_text(model,temperature,start_string,num_generate):
  #convert string to nums (vectorizing)
  input_val=[char2idx[c] for c in start_string] 
  input_val=tf.expand_dims(input_val,0)
  text_generated=[]
  
  model.reset_states()
  for i in range(num_generate):
    predict=model(input_val)
    predict=tf.squeeze(predict,0)
    predict/=temperature
    
    predict_id=tf.random.categorical(predict,num_samples=1)[-1,0].numpy()
    input_val=tf.expand_dims([predict_id],0) 
    text_generated+=[idx2char[predict_id]]

  return start_string+''.join(text_generated)

# Parameters

In [0]:
#Parameters
#Auto

paras=[-1,0.001,100,128,512,1024,100,0]
alter=[-1,0.002,1000,256,256,512,200,1]

for i in range(0):
    parameters=paras[:]
    parameters[i]=alter[i]
    lr,seq_len,batch_size,embedding_dim,lstm_units,epochs,data_chuck=parameters[1:]
    
    checkpoint_filepath=get_checkpoints(lr, seq_len, batch_size, embedding_dim, lstm_units, epochs, data_chuck);print(checkpoint_filepath)
    call_model(lr,seq_len,batch_size,embedding_dim,lstm_units,epochs,data_chuck)
    



In [0]:
#Parameters
#Manually

paras=[0.003,100,128,512,1024,100,1,0.6,160]
lr,seq_len,batch_size,embedding_dim,lstm_units,epochs,data_chuck,temperature,num_generate=paras
checkpoint_filepath=get_checkpoints(lr, seq_len, batch_size, embedding_dim, lstm_units, epochs, data_chuck);print(checkpoint_filepath)
call_model(lr,seq_len,batch_size,embedding_dim,lstm_units,epochs,data_chuck)
start_string="段譽卻仍是抬起了頭望著她，見那少女雙腳蕩啊蕩的，似乎這麼坐樑上甚是好玩，問道：“是你救我的麼？"
call_predict(embedding_dim,lstm_units,checkpoint_filepath,temperature,num_generate,start_string)


paras=[0.003,100,128,512,1024,100,0,0.6,160]
lr,seq_len,batch_size,embedding_dim,lstm_units,epochs,data_chuck,temperature,num_generate=paras
checkpoint_filepath=get_checkpoints(lr, seq_len, batch_size, embedding_dim, lstm_units, epochs, data_chuck);print(checkpoint_filepath)
call_model(lr,seq_len,batch_size,embedding_dim,lstm_units,epochs,data_chuck)
start_string="段譽卻仍是抬起了頭望著她，見那少女雙腳蕩啊蕩的，似乎這麼坐樑上甚是好玩，問道：“是你救我的麼？"
call_predict(embedding_dim,lstm_units,checkpoint_filepath,temperature,num_generate,start_string)

./train/0.003seq100batch128_embed512unit1024_epoch100_1shuffle/checkpoint
Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_17 (Embedding)     (128, None, 512)          2249216   
_________________________________________________________________
lstm_17 (LSTM)               (128, None, 1024)         6295552   
_________________________________________________________________
dense_17 (Dense)             (128, None, 4393)         4502825   
Total params: 13,047,593
Trainable params: 13,047,593
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
82/97 [========================>.....] - ETA: 4s - loss: 5.8390

InvalidArgumentError: ignored

# Measurements & Visualizations

In [0]:
%tensorboard --logdir logs/